# Добування бажаних бізнес оглядів 
В попередньому зошиті ми досягли двох речей. 
По-перше, ми визначили категорії бізнесу (italian / Pizza), на які буде спрямована наша кампанія. 
По-друге, ми визначили ідентифікатори підприємств (business ID), пов’язані з цими категоріями. Знання business ID дозволяє вибрати  з файлу відгуків лише ті відгуки, які нас цікавлять. Перш ніж поспішати читати дані оглядів в один великий фрейм даних, щоб потім відфільтрувати лише ті ідентифікатори, які вас цікавлять, ви, як Data Scientist, спочатку  повинні отримати уявлення про розмір файлу. Файл оглядів досить великий (майже шість мільйонів рядків). Якщо у вас дуже хороший комп'ютер з великим обсягом оперативної пам’яті, або ви чомусь хочете запустити монітор системи / пам’яті і спостерігати, як вільна пам'ять падає все далі і далі, поки комп'ютер не захопить її всю, вам варто замислитися про те, як читати лише ті рядки, які нас дійсно цікавлять. Таким чином, ми будемо споживати лише мінімум необхідної оперативної пам’яті. Звичайно, загалом, навіть у цьому випадку нам  слід робити якийсь розрахунок, чи відповідає файл можливостям комп'ютера. У цьому випадку слід мати подвійний запас ресурсів для більшості сучасних комп'ютерів. Завдання цього зошита - проаналізувати файл даних з оглядами та зберегти (набагато меншу) підмножину даних, які нас власне цікавлять.

In [1]:
# import the required libraries here
import pandas as pd
None

##  Список необхідних  business IDs
Спочатку нам потрібно прочитати  список  business ID, який ми отримали раніше.

In [2]:
# task: read in our previously created file that contains the business IDs of interest
# one line
businesses = pd.read_csv('italian_pizza.csv')

Важливо спочатку  зробити швидку перевірку зчитаних даних.

In [3]:
businesses.head()

,business_id,review_count,stars,state
0,fweCYi8FmbJXHCqLnwuk8w,16,4.0,OH
1,PZ-LZzSlhSe9utkQYU8pFg,40,4.0,NV
2,BvYU3jvGd0TJ7IyZdfiN2Q,16,3.5,NC
3,PkDghu4aan2_wxrhXjTEgg,16,3.5,AB
4,RFbMVekR8lU9tPJ8sWrwHg,18,4.0,ON


Ми хочемо відобразити лише колонку business_id як список. Варто також зробити подвійну перевірку, чи відповідає він стовпцю business_id, наведеному вище.

In [4]:
business_ids = businesses['business_id'].values
print(business_ids[:5])

['fweCYi8FmbJXHCqLnwuk8w' 'PZ-LZzSlhSe9utkQYU8pFg'
 'BvYU3jvGd0TJ7IyZdfiN2Q' 'PkDghu4aan2_wxrhXjTEgg'
 'RFbMVekR8lU9tPJ8sWrwHg']


## Фільтрування великого файлу даних фрагментами
Файл даних з оглядами містить близько 6 мільйонів рядків. Нам немає потреби використовувати всі ці рядки. Наш загальний підхід полягає в тому, щоб прочитати файл і перевірити business_id у кожному рядку. Якщо цей ідентифікатор міститься в нашому списку розшукуваних ідентифікаторів, ми збережемо цей рядок. Це буде найшвидше, але найдорожче для пам'яті, за один раз прочитати весь файл з диску, а потім фільтрувати рядки в пам'яті. З іншого боку, найефективніше з точки зору використання пам’яті читати по одному запису за один раз, зразу ж перевіряти, чи потрібний нам цей рядок (і відкинути, якщо ні). Такий підхід буде найповільнішим. Ми можемо досягти балансу, коли будемо вибирати достатньо великий фрагмент, який все ще легко вписується в пам'ять, але при цьому нам не потрібно читати занадто багато фрагментів.

In [5]:
# For convenience here, again we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple here.
# If you're comfortable specifying a filepath to files outside of this working directory, then feel free to do so.
# task: create a reader object for the review json file
# Hint: use lines=True as before but add the chunksize=100000 parameter
# one line of code here
review_reader = pd.read_json('JSON/review.json', lines=True, chunksize=100000)

Ми використовуємо магію часу для наступної комірки, не цікавлячись тим, скільки часу це потребуватиме. Якщо у вас є час і вам цікаво, ви можете поекспериментувати з розміром фрагменту і контролювати використання системної пам'яті та час, який займає робота цієї комірки. Я пропоную вам зберегти свій зошит перед запуском цієї комірки, про всяк випадок.

In [10]:
%%time
# task: process the file one chunk at a time,
# filter that chunk for rows with a business_id in business_ids
# You can either do this within in a loop, having initialized an empty list,
# or using a more pythonic list comprehension
reviews = [r for chunk in review_reader for r in chunk[chunk['business_id'].apply(lambda x: x in business_ids)].values]
# (this took some 24 minutes on my old i7)

Wall time: 16min 27s


Після успішного виконання коду у вас є прочитані бажані огляди. Але ми ще не маємо їх у зручній DataFrame:

In [11]:
type(reviews)

list

In [12]:
len(reviews)

578356

Перетворіть свої огляди в структуру DataFrame. 
Підказка: тут ви можете скористатися методом concat бібліотеки pandas.

In [13]:
# task convert your reviews into a DataFrame
# one line of code here
reviews = pd.DataFrame(reviews)

In [14]:
reviews.shape

(578356, 9)

In [15]:
reviews.head()

,0,1,2,3,4,5,6,7,8
0,MQ2UnmF6aNtbK3OzFo6ayw,1WDC-kk2aQi7HIyASCeFVA,5dy8OiWJty9AuxBcBzeBZQ,3,1,0,1,"Overall, the ambiance was romantic with the di...",2018-01-26 05:16:34
1,H1tkqDvXqYhMzvOc2KzKyw,qqmDaJ663juooUqdkJWS5w,oD8kSlXINk5u7d6HQh2SxQ,4,0,0,0,Waiter was very nice and the food menu had nic...,2017-07-31 18:39:57
2,1uQ6jFOPCy0OTZ3Eew0nCg,Ws4rMCXBe1beylwkMyHwog,obzBY9Pea6nUXsjtEjo-Cw,2,4,0,1,Where to begin...\n\nWe went to le Bistro one ...,2011-07-08 21:09:36
3,b685eEc2s88WnB-crlZW0g,I9IqqJYtRsamLZ8S_yewCA,83cVpTJNyzv9lJ20vzBgdg,5,0,0,0,We ordered 2 18in cheese pizzas for delivery. ...,2016-08-28 23:53:24
4,DOQM7XeMuoAEiB7E1PmUeg,9_PsmeKSuJ2XGvz-Snfhhg,SbqlNhbtfW6Wyj61vyWxUg,1,0,0,0,Worst service available even for an airport lo...,2016-12-24 23:25:39


## Збереження даних 
Зробивши всю цю важку роботу, фільтруючи файл оглядів і задокументувавши процес у цьому зошиті, нам потрібно зберегти дані. 
Збережемо огляди структури DataFrame на csv, який назвемо "reviews_filtered.csv".

In [16]:
# task: save the DataFrame to the specified file now.
# don't forget to use index=False
reviews.to_csv(path_or_buf='reviews_filtered.csv', index=False)

Якщо поглянути на розмір оригінального файлу json та наш новий файл csv, побачимо, що ми перейшли від 4,4 ГБ до 325 МБ. Це набагато зручніше!

# Підсумок

У цьому проекті ви мали навчитися, як вирішити проблему та визначити відповідні дані, використовувати дані, щоб отримати уявлення про проблему та прийняти рішення, а потім використовувати ці знання для отримання потрібної підмножини даних з надзвичайно великого файлу. У наступному зошиті ми зануримось у ці дані.